# 🩲 Memory-DFT: Quick Start

**H-CSP/Λ³理論に基づく履歴依存密度汎関数理論**

🩲→🧪→Λ³

## 1. セットアップ

In [ ]:
# GitHubからクローン
!git clone https://github.com/miosync-masa/lambda3-memory-dft.git

# パスを通す（リポジトリのルートを追加）
import sys
sys.path.insert(0, '/content/lambda3-memory-dft')

# または pip install
# %cd /content/lambda3-memory-dft
# !pip install -e .

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from memory_dft import (
    SparseHamiltonianEngine,
    TimeEvolutionEngine,
    EvolutionConfig,
    CompositeMemoryKernel,
    KernelWeights,
    Lambda3Calculator,
    HCSPValidator
)

print("✅ Memory-DFT loaded!")

## 2. 簡単なテスト: 2サイトHeisenbergモデル

In [ ]:
# 2サイト系（H2分子の簡易モデル）
engine = SparseHamiltonianEngine(n_sites=2, use_gpu=False)

# ハミルトニアン
bonds = [(0, 1)]
H_K, H_V = engine.build_heisenberg_hamiltonian(bonds, J=1.0, Jz=0.5)

# 初期状態: |↑↓⟩ + |↓↑⟩
psi0 = np.array([0, 1, 1, 0], dtype=np.complex128)
psi0 = psi0 / np.linalg.norm(psi0)

print(f"System: {engine.n_sites} sites, dim={engine.dim}")

## 3. Memory-DFT時間発展

In [ ]:
# Memory-DFT設定
config = EvolutionConfig(
    t_end=20.0,
    dt=0.1,
    use_memory=True,
    memory_strength=0.1,  # η=10%
    verbose=True
)

# 発展エンジン
evol = TimeEvolutionEngine(H_K, H_V, config, use_gpu=False)

# 実行
result = evol.run(psi0)

## 4. Λ軌跡のプロット

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Λ軌跡
axes[0].plot(result.times, result.lambdas, 'b-', linewidth=1.5)
axes[0].axhline(y=1.0, color='r', linestyle='--', label='Critical Λ=1')
axes[0].set_xlabel('Time')
axes[0].set_ylabel('Λ')
axes[0].set_title('Λ Trajectory (Memory-DFT)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# エネルギー
axes[1].plot(result.times, result.energies, 'g-', linewidth=1.5)
axes[1].set_xlabel('Time')
axes[1].set_ylabel('Energy')
axes[1].set_title('Energy Conservation')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nEnergy drift: ΔE = {abs(result.energies[-1] - result.energies[0]):.6f}")

## 5. H-CSP公理の検証

In [ ]:
validator = HCSPValidator()
validation = validator.validate_all(result.lambdas)

print("H-CSP Axiom Validation")
print("="*50)

for axiom, check in validation.items():
    print(f"\n{axiom}:")
    for k, v in check.items():
        print(f"  {k}: {v}")

## 6. Memory-DFT vs 標準量子力学の比較

In [ ]:
# 標準QM
config_std = EvolutionConfig(
    t_end=20.0,
    dt=0.1,
    use_memory=False,
    verbose=False
)

evol_std = TimeEvolutionEngine(H_K, H_V, config_std, use_gpu=False)
result_std = evol_std.run(psi0)

# 比較プロット
plt.figure(figsize=(10, 5))
plt.plot(result.times, result.lambdas, 'b-', label='Memory-DFT (η=0.1)', linewidth=2)
plt.plot(result_std.times, result_std.lambdas, 'r--', label='Standard QM', linewidth=2)
plt.axhline(y=1.0, color='gray', linestyle=':', alpha=0.5)
plt.xlabel('Time')
plt.ylabel('Λ')
plt.title('Memory-DFT vs Standard QM')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# 差分
diff = np.array(result.lambdas) - np.array(result_std.lambdas)
print(f"Max |ΔΛ|: {np.max(np.abs(diff)):.4f}")
print(f"Mean |ΔΛ|: {np.mean(np.abs(diff)):.4f}")

## 7. Memory Kernel成分の可視化

In [ ]:
kernel = CompositeMemoryKernel(
    weights=KernelWeights(field=0.5, phys=0.3, chem=0.2),
    gamma_field=1.0,
    beta_phys=0.5,
    tau0_phys=10.0,
    t_react_chem=5.0
)

# 履歴時刻
history_times = np.arange(0, 30, 0.5)
t_current = 30.0

# 各成分の重み
decomp = kernel.decompose(t_current, history_times)

plt.figure(figsize=(10, 5))
plt.plot(history_times, decomp['field'], 'b-', label='Field (power-law)', linewidth=2)
plt.plot(history_times, decomp['phys'], 'g-', label='Phys (stretched exp)', linewidth=2)
plt.plot(history_times, decomp['chem'], 'r-', label='Chem (step)', linewidth=2)
plt.plot(history_times, decomp['total'], 'k--', label='Total', linewidth=2)
plt.xlabel('τ (history time)')
plt.ylabel('Weight')
plt.title('Memory Kernel Decomposition (H-CSP Environment Hierarchy)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(kernel)

## 🎉 完了！

Memory-DFTの基本動作を確認しました。

次のステップ:
- より大きな系でのテスト
- PySCFとの連携（γ計算）
- GPU加速（CuPy）